In [1]:
import numpy as np
import random
import time
import torch
from x_transformers.x_transformers import XTransformer
import torch

from run_experiment import *
from generate_data import *

## Variables

In [5]:
from sklearn.model_selection import ParameterGrid

TAG = 'improve_score_2paper_55len'

TASK_NAME = 'copy'
TRAIN_SIZE = 100_000
VAL_SIZE = 2_000
TEST_SIZE = 10_000
NUM_INITS = 3


NUM_BATCHES = int(2.3e5)
BATCH_SIZE = 64
LEARNING_RATE = 3e-4
GENERATE_EVERY  = 3000
NUM_TOKENS = 16 + 2
ENC_SEQ_LEN = 55
DEC_SEQ_LEN = 110

INPUT_LEN = 55

model_parameters = ParameterGrid({'dim': [128],
    'tie_token_embeds': [True],
    'return_tgt_loss': [True],
    'enc_num_tokens': [NUM_TOKENS],
    'depth,heads': [(2,4)],
    'enc_max_seq_len': [55],
    'dec_num_tokens': [NUM_TOKENS],
    'dec_max_seq_len': [DEC_SEQ_LEN],
    'enc_num_memory_tokens': [0]})

print('Total runs: ', NUM_INITS * len(model_parameters))

Total runs:  3


In [3]:
# for i, p in enumerate(model_parameters):
#     print(i, p)

#### Generate data

In [4]:
# class copy_generator:
#     def __init__(self):
#         self.src_mask = torch.ones(BATCH_SIZE, ENC_SEQ_LEN).bool().cuda()
#         self.tgt_mask = torch.ones(BATCH_SIZE, DEC_SEQ_LEN+1).bool().cuda()
    
#     def __next__(self):
#         X = np.zeros([BATCH_SIZE, ENC_SEQ_LEN]).astype(int)
#         y = np.zeros([BATCH_SIZE, DEC_SEQ_LEN+1]).astype(int)
#         y[:, 0] = 1
#         for i in range(BATCH_SIZE):
#             sequence_length = np.random.randint(1, ENC_SEQ_LEN)
#             random_sequence = np.random.randint(2, NUM_TOKENS, sequence_length)
            
#             X[i, :sequence_length] = random_sequence
#             y[i, 1: 2 * sequence_length + 1] = np.concatenate([random_sequence] * 2)

#         return torch.tensor(X), torch.tensor(y), self.src_mask, self.tgt_mask      
    
# generator = copy_generator()
# generate_data(generator, task_name=TASK_NAME, train_size=TRAIN_SIZE, test_size=TEST_SIZE, val_size=VAL_SIZE)  

#### Gridesarch params

In [6]:
optimizer = torch.optim.Adam

optim_params = list(ParameterGrid({
    'lr': [0.001, 0.0008, 0.0012]
}))

print(len(optim_params))

3


In [10]:
optim_params

[{'lr': 0.001}, {'lr': 0.0008}, {'lr': 0.0012}]

In [ ]:
# gen_train = data_loader(task_name=f'{TASK_NAME}_train', batch_size=BATCH_SIZE, enc_seq_len=INPUT_LEN, dec_seq_len=DEC_SEQ_LEN)
# gen_val = data_loader(task_name=f'{TASK_NAME}_val', batch_size=VAL_SIZE, enc_seq_len=INPUT_LEN, dec_seq_len=DEC_SEQ_LEN)
# gen_test = data_loader(task_name=f'{TASK_NAME}_test', batch_size=TEST_SIZE, enc_seq_len=INPUT_LEN, dec_seq_len=DEC_SEQ_LEN)


# print_file = f'logs/{TASK_NAME}_{TAG}_cout_logs.txt'
# t = time.time()

# param = list(model_parameters)[0]
# param['enc_depth'], param['enc_heads'] = param['depth,heads']
# param['dec_depth'], param['dec_heads'] = param['depth,heads']
# param.pop('depth,heads')

# with torch.cuda.device(0):
#     for i, optim_param in enumerate(list(optim_params)):
#         with open(print_file, 'a') as f:
#             f.write('\n\n' + str(optim_param)+'\n')
        
#         for init_num in range(1):
#             model = XTransformer(**param).cuda()

#             model_name = f"{TASK_NAME}{INPUT_LEN}_dim{param['dim']}d{param['enc_depth']}h{param['enc_heads']}M{param['enc_num_memory_tokens']}l{param['enc_max_seq_len']}_v{init_num}_{optim_param}"

#             optim = optimizer(model.parameters(), **optim_param)
#             train_validate_model(model, 
#                                 train_generator=gen_train, 
#                                 val_generator=gen_val, 
#                                 optim=optim, 
#                                 model_name=model_name, 
#                                 dec_seq_len=DEC_SEQ_LEN,
#                                 num_batches=NUM_BATCHES,
#                                 generate_every=GENERATE_EVERY,
#                                 print_file=print_file)
#             test_model(model, gen_test, model_name, param, TASK_NAME, tag=str(optim_param), dec_seq_len=param['dec_max_seq_len'])
#             with open(print_file, 'a') as f:
#                 f.write(f'\nTotal time: {time.time() - t}\n')
#             t = time.time()

### Run

In [7]:
LEARNING_RATE = 0.001

model_parameters = ParameterGrid({'dim': [128],
    'tie_token_embeds': [True],
    'return_tgt_loss': [True],
    'enc_num_tokens': [NUM_TOKENS],
    'depth,heads': [(2,4)],
    'enc_max_seq_len': [55, 28],
    'dec_num_tokens': [NUM_TOKENS],
    'dec_max_seq_len': [DEC_SEQ_LEN],
    'enc_num_memory_tokens': [0, 4, 16]})

print('Total runs: ', NUM_INITS * len(model_parameters))

Total runs:  18


In [ ]:
gen_train = data_loader(task_name=f'{TASK_NAME}_train', batch_size=BATCH_SIZE, enc_seq_len=INPUT_LEN, dec_seq_len=DEC_SEQ_LEN)
gen_val = data_loader(task_name=f'{TASK_NAME}_val', batch_size=VAL_SIZE, enc_seq_len=INPUT_LEN, dec_seq_len=DEC_SEQ_LEN)
gen_test = data_loader(task_name=f'{TASK_NAME}_test', batch_size=TEST_SIZE, enc_seq_len=INPUT_LEN, dec_seq_len=DEC_SEQ_LEN)


t = time.time()
with torch.cuda.device(0):
    for init_num in range(NUM_INITS):
        print('\n\n\nInit number ', init_num)
        for i, param in enumerate(list(model_parameters)):
            print(param)
            param['enc_depth'], param['enc_heads'] = param['depth,heads']
            param['dec_depth'], param['dec_heads'] = param['depth,heads']
            param.pop('depth,heads')

            print(i / len(model_parameters) * 100, '%')
            model = XTransformer(**param).cuda()

            model_name = f"{TASK_NAME}{INPUT_LEN}_dim{param['dim']}d{param['enc_depth']}h{param['enc_heads']}M{param['enc_num_memory_tokens']}l{param['enc_max_seq_len']}_v{init_num}"

            optim = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
            train_validate_model(model, 
                                train_generator=gen_train, 
                                val_generator=gen_val, 
                                optim=optim, 
                                model_name=model_name, 
                                dec_seq_len=DEC_SEQ_LEN,
                                num_batches=NUM_BATCHES,
                                generate_every=GENERATE_EVERY)
            test_model(model, gen_test, model_name, param, TASK_NAME, tag=TAG, dec_seq_len=param['dec_max_seq_len'])
            print('Total time: ', time.time() - t)
            t = time.time()




Init number  0
{'dec_max_seq_len': 110, 'dec_num_tokens': 18, 'depth,heads': (2, 4), 'dim': 128, 'enc_max_seq_len': 55, 'enc_num_memory_tokens': 0, 'enc_num_tokens': 18, 'return_tgt_loss': True, 'tie_token_embeds': True}
0.0 %


stopped on: {'dec_max_seq_len': 32, 'dec_num_tokens': 18, 'depth,heads': (2, 4), 'dim': 64, 'enc_max_seq_len': 16, 'enc_num_memory_tokens': 16, 'enc_num_tokens': 18, 'return_tgt_loss': True, 'tie_token_embeds': True}

### Test!

In [ ]:
init_num = 0

gen_train = data_loader(task_name=f'{TASK_NAME}_train', batch_size=BATCH_SIZE, enc_seq_len=ENC_SEQ_LEN, dec_seq_len=DEC_SEQ_LEN)
gen_val = data_loader(task_name=f'{TASK_NAME}_val', batch_size=VAL_SIZE, enc_seq_len=ENC_SEQ_LEN, dec_seq_len=DEC_SEQ_LEN)
gen_test = data_loader(task_name=f'{TASK_NAME}_test', batch_size=TEST_SIZE, enc_seq_len=ENC_SEQ_LEN, dec_seq_len=DEC_SEQ_LEN)


param = list(model_parameters)[5]
print(param)
param['enc_depth'], param['enc_heads'] = param['depth,heads']
param['dec_depth'], param['dec_heads'] = param['depth,heads']
param.pop('depth,heads')

model = XTransformer(**param).cuda()

model_name = f"{TASK_NAME}_dim{param['dim']}d{param['enc_depth']}h{param['enc_heads']}M{param['enc_num_memory_tokens']}l{param['enc_max_seq_len']}_v{init_num}"

optim = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

src, tgt, _, _ = next(gen_train)

print(model.encoder.max_seq_len, model.encoder.num_memory_tokens)
model.encoder(torch.cat((src, src)), return_embeddings=True).shape